## Galaxy morphologyカタログ（Hubble系列8クラス）

In [1]:
import os, shutil
import sys, time
import numpy as np
import pandas as pd
from astropy.io import fits

# 作業用ディレクトリ定義（要カスタマイズ）
galaxy_data_directory = '/home/satoshi/Galaxy/'

# Galaxy Zoo 2カタログ
#http://gz2hart.s3.amazonaws.com/gz2_hart16.fits.gzからダウンロード&解凍し、galaxy_data_directory配下にコピー
gz2_catalog = galaxy_data_directory  + 'gz2_hart16.fits'

# Galaxy Zoo 2メタデータ
# http://zooniverse-data.s3.amazonaws.com/galaxy-zoo-2/gz2sample.fits.gzからダウンロード&解凍し、galaxy_data_directory配下にコピー
gz2_metadata = galaxy_data_directory + 'gz2sample.fits'

# ワーク用カタログ
gz2_catalog_work = galaxy_data_directory + 'gz2_catalog_work.fits'
if os.path.exists(gz2_catalog_work):
    os.remove(gz2_catalog_work)

# 新カテゴリカタログ
gz2_catalog_v2 = galaxy_data_directory + 'gz2_catalog_hubble_8class.fits'


#### 'Galaxy Zoo 2'オリジナルのカタログにメタデータ用の空列を追加したワークカタログを作成する

In [2]:
with fits.open(gz2_catalog) as hdul:
    data = hdul[1].data
    print(len(data))
    orig_cols = data.columns
    add_cols = fits.ColDefs([
        fits.Column(name = 'HUBBLE_CLASS', format = '8A',
                    array = np.empty(len(data))),
        fits.Column(name = 'PETROR90_R', format = 'E', 
                   array = np.empty(len(data))), 
        fits.Column(name = 'PETROR50_R_KPC_SIMPLE_BIN', format = 'I',
                   array = np.empty(len(data))),
        fits.Column(name = 'PETROMAG_MR_SIMPLE_BIN', format = 'I',
                   array = np.empty(len(data))),
        fits.Column(name = 'REDSHIFT_SIMPLE_BIN', format = 'I',
                   array = np.empty(len(data))),
        fits.Column(name = 'FILE_NAME', format = '64A',
                    array = np.empty(len(data)))
    ])
    hdul = fits.BinTableHDU.from_columns(orig_cols + add_cols)
    hdul.writeto(gz2_catalog_work) 
    

239695


#### 'Galaxy Zoo 2'メタデータを読み込む

In [3]:
with fits.open(gz2_metadata) as hdul:
    data = hdul[1].data
    print(len(data))
    redshift_bin = np.array(data.field('REDSHIFT_SIMPLE_BIN'))
    obj_id = np.array(data.field('OBJID'))
    petroR90_r = np.array(data.field('PETROR90_R'))
    mgr_bin = np.array(data.field('PETROMAG_MR_SIMPLE_BIN'))
    petroR50_bin= np.array(data.field('PETROR50_R_KPC_SIMPLE_BIN'))
    
    redshift_bin_series = pd.Series(redshift_bin, index= obj_id)
    petroR90_r_series = pd.Series(petroR90_r, index =  obj_id)
    mgr_bin_series = pd.Series(mgr_bin, index =  obj_id)
    petroR50_bin_series = pd.Series(petroR50_bin, index =  obj_id)
       

325704


#### オリジナルカタログのクラス分類をHubble系列8クラスに集約し、メタデータを追加して、新カタログを作成する

In [4]:
with fits.open(gz2_catalog_work) as hdul:
    data = hdul[1].data
    for i in range(len(data)):    
        # Er, Ei, Ec
        if data.field('gz2_class')[i][:1] == 'E':
            if data.field('gz2_class')[i][:2] == 'Er':
                # Er => E0
                data.field('HUBBLE_CLASS')[i] = 'E0'
            elif data.field('gz2_class')[i][:2] == 'Ei':
                # Ei => E3
                data.field('HUBBLE_CLASS')[i] = 'E3'
            else:
                # Ec => E7 
                data.field('HUBBLE_CLASS')[i] = 'E7'
        # Ser, Seb, Sen => Edgeonへ統合 
        elif data.field('gz2_class')[i][:3] == 'Ser' or data.field('gz2_class')[i][:3] == 'Seb' \
                                                                            or data.field('gz2_class')[i][:3] == 'Sen':
            data.field('HUBBLE_CLASS')[i] = 'Edgeon'
                
       
        # SBd, SBc, SBb, SBa
        elif data.field('gz2_class')[i][:3] == 'SBd' or data.field('gz2_class')[i][:3] == 'SBc' \
                                                                            or data.field('gz2_class')[i][:3] == 'SBb' \
                                                                            or data.field('gz2_class')[i][:3] == 'SBa':
            # spiralがあるか？ => SBabc
            if 't' in data.field('gz2_class')[i] or 'm' in data.field('gz2_class')[i] or  'l' in data.field('gz2_class')[i]:
            # SBd, SBc, SBb, SBa => SBa
                data.field('HUBBLE_CLASS')[i] = 'SBabc'
            # spiralなし => SB0
            else:
                # SBd, SBc, SBb, SBa => SB0
                data.field('HUBBLE_CLASS')[i] = 'SB0'
                
        # Sd, Sc, Sb, Sa 
        elif data.field('gz2_class')[i][:2] == 'Sd' or data.field('gz2_class')[i][:2] == 'Sc' \
                                                                                  or data.field('gz2_class')[i][:2] == 'Sb' \
                                                                                  or data.field('gz2_class')[i][:2] == 'Sa':
            
            # spiralがあるか？ => SAabc
            if 't' in data.field('gz2_class')[i] or 'm' in data.field('gz2_class')[i] or  'l' in data.field('gz2_class')[i]:
            # Sd, Sc, Sb, Sa => Sabc
                data.field('HUBBLE_CLASS')[i] = 'Sabc'
            # spiralなし => SA0
            else:
                # Sd, Sc, Sb, Sa => S0
                data.field('HUBBLE_CLASS')[i] = 'S0'
            
        # not galaxy
        elif data.field('gz2_class')[i][:1] == 'A':
            pass
        else:
            print('undefined category', i, data.field('gz2_class')[i])

        # メタデータ項目を取り込む
        data.field('PETROR90_R')[i] =  petroR90_r_series[data.field('dr7objid')[i]]
        data.field('PETROR50_R_KPC_SIMPLE_BIN')[i] =  petroR50_bin_series[data.field('dr7objid')[i]]
        data.field('PETROMAG_MR_SIMPLE_BIN')[i] =  mgr_bin_series[data.field('dr7objid')[i]]
        data.field('REDSHIFT_SIMPLE_BIN')[i] =  redshift_bin_series[data.field('dr7objid')[i]]
        
        # ファイル名セット
        fname = data.field('HUBBLE_CLASS')[i] + '_' + str(data.field('REDSHIFT_SIMPLE_BIN')[i]) + '_' \
                                                                                   + str(int(data.field('PETROR90_R')[i])) + '_' \
                                                                                   + str(data.field('PETROMAG_MR_SIMPLE_BIN')[i]) + '_' \
                                                                                   + str(data.field('dr7objid')[i]) + '.jpeg'
        data.field('FILE_NAME')[i] = fname
        
    # not galaxyのrowを削除
    mask= data.field('gz2_class') != 'A'
    # クラス名を集約したカタログを作成
    newdata=data[mask]
    hdul=fits.BinTableHDU(data=newdata)
    hdul.writeto(gz2_catalog_v2) 

print(gz2_catalog_v2, '  completed')
    

/home/satoshi/Galaxy/gz2_catalog_hubble_8class.fits   completed


#### クラス構成の確認

In [5]:
with fits.open(gz2_catalog_v2) as hdul:
    data = hdul[1].data
    
    print('category_catalog rows = ', data.shape[0])
    galaxy_class=np.array(data.field('HUBBLE_CLASS'))
    
    galaxy_series=pd.Series(galaxy_class)
    unique_galaxy_class = sorted(galaxy_series.unique()) 
    print(unique_galaxy_class, '  ', len(unique_galaxy_class))
    
    for class_name in unique_galaxy_class:
        galaxy_list_series=pd.Series(np.array([i for i in range(len(data)) if galaxy_class[i] == class_name]))
        class_ratio = len(galaxy_list_series) / data.shape[0] * 100
        print(class_name, '  total  ', len(galaxy_list_series), '  ', int(class_ratio), '%')
           

category_catalog rows =  239100
['E0', 'E3', 'E7', 'Edgeon', 'S0', 'SB0', 'SBabc', 'Sabc']    8
E0   total   40015    16 %
E3   total   47166    19 %
E7   total   10489    4 %
Edgeon   total   19606    8 %
S0   total   18209    7 %
SB0   total   5023    2 %
SBabc   total   40558    16 %
Sabc   total   58034    24 %
